<a href="https://colab.research.google.com/github/Mauriciomsonic/laboratoriomsonicA.I./blob/main/Desafio_ML_Criando_um_sistema_de_assist%C3%AAncia_virtual_do_zero_comando_de_voz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalar bibliotecas necessárias
!pip install gtts SpeechRecognition wikipedia

import speech_recognition as sr
import wikipedia
import webbrowser
from gtts import gTTS
from IPython.display import Audio, display, HTML
import os
import requests
from urllib.parse import quote
from base64 import b64encode
from google.colab import files
import io

# Configurar wikipedia em português
wikipedia.set_lang("pt")

# Função de texto para áudio
def text_to_speech(text, filename="response.mp3"):
    try:
        tts = gTTS(text=text, lang='pt', slow=False)
        tts.save(filename)

        # Converter para base64 para embedar no HTML
        with open(filename, 'rb') as f:
            audio_data = f.read()
        audio_base64 = b64encode(audio_data).decode('utf-8')
        audio_html = f'<audio controls autoplay><source src="data:audio/mp3;base64,{audio_base64}" type="audio/mp3"></audio>'
        display(HTML(audio_html))

        return True
    except Exception as e:
        print(f"Erro no TTS: {e}")
        return False

# Função de áudio para texto via upload
def speech_to_text_upload():
    print("🎤 Por favor, faça upload de um arquivo de áudio (.wav ou .mp3)")
    print("📝 Grave seu comando primeiro e depois faça o upload")

    uploaded = files.upload()

    if not uploaded:
        text_to_speech("Nenhum arquivo foi enviado. Tente novamente.")
        return None

    filename = list(uploaded.keys())[0]
    recognizer = sr.Recognizer()

    try:
        with sr.AudioFile(io.BytesIO(uploaded[filename])) as source:
            print("🔍 Processando áudio...")
            audio = recognizer.record(source)
            command = recognizer.recognize_google(audio, language='pt-BR')
            print(f"📝 Comando reconhecido: {command}")
            return command.lower()

    except sr.UnknownValueError:
        text_to_speech("Desculpe, não consegui entender o áudio. Pode gravar novamente?")
        return None
    except sr.RequestError:
        text_to_speech("Erro no serviço de reconhecimento de voz")
        return None
    except Exception as e:
        text_to_speech(f"Erro ao processar o áudio: {str(e)}")
        return None

# Função alternativa: entrada de texto manual
def text_input_fallback():
    print("📝 Digite seu comando manualmente:")
    command = input().strip().lower()
    return command if command else None

# Função para pesquisar no Google
def search_google(query):
    try:
        search_url = f"https://www.google.com/search?q={quote(query)}"
        webbrowser.open(search_url)
        return f"Pesquisando no Google por: {query}"
    except Exception as e:
        return f"Erro ao abrir Google: {str(e)}"

# Função para pesquisar na Wikipedia
def search_wikipedia(query):
    try:
        result = wikipedia.summary(query, sentences=2)
        return f"Segundo a Wikipedia: {result}"
    except wikipedia.exceptions.DisambiguationError:
        return "Encontrei muitos resultados. Por favor, seja mais específico."
    except wikipedia.exceptions.PageError:
        return "Não encontrei resultados na Wikipedia."
    except Exception as e:
        return f"Erro na pesquisa: {str(e)}"

# Função para encontrar Carrefour mais próximo (simulação)
def find_nearest_carrefour():
    locations = [
        "Shopping Iguatemi - 2km daqui",
        "Bourbon Shopping - 3km daqui",
        "Shopping Total - 5km daqui",
        "ParkShopping São Caetano - 8km daqui"
    ]
    return f"O Carrefour mais próximo fica no {locations[0]}"

# Função para abrir YouTube
def open_youtube():
    try:
        webbrowser.open("https://www.youtube.com")
        return "Abrindo YouTube"
    except Exception as e:
        return f"Erro ao abrir YouTube: {str(e)}"

# Função para abrir pesquisa de curso de IA
def search_ai_course():
    try:
        search_url = "https://www.google.com/search?q=curso+de+inteligência+artificial"
        webbrowser.open(search_url)

        # Também abre Wikipedia
        wiki_url = "https://pt.wikipedia.org/wiki/Inteligência_artificial"
        webbrowser.open(wiki_url, new=2)

        return "Abrindo pesquisas sobre curso de inteligência artificial no Google e Wikipedia"
    except Exception as e:
        return f"Erro ao abrir pesquisas: {str(e)}"

# Sistema principal de assistência virtual
def virtual_assistant():
    text_to_speech("Olá! Sou sua assistente virtual. Vamos começar!")

    while True:
        print("\n" + "="*60)
        print("🎯 ESCOLHA UMA OPÇÃO:")
        print("1. 🎤 Upload de arquivo de áudio")
        print("2. ⌨️  Digitar comando manualmente")
        print("3. ❌ Sair")
        print("="*60)

        choice = input("Digite sua escolha (1, 2 ou 3): ").strip()

        if choice == "3":
            text_to_speech("Até logo! Foi um prazer ajudar.")
            break

        elif choice == "1":
            command = speech_to_text_upload()
        elif choice == "2":
            command = text_input_fallback()
        else:
            print("Opção inválida. Tente novamente.")
            continue

        if not command:
            continue

        # Processar comandos
        if "sair" in command or "parar" in command:
            text_to_speech("Até logo! Foi um prazer ajudar.")
            break

        elif "youtube" in command:
            response = open_youtube()
            text_to_speech(response)

        elif "carrefour" in command or "hipermercado" in command:
            response = find_nearest_carrefour()
            text_to_speech(response)

        elif "inteligência artificial" in command or "curso ia" in command:
            response = search_ai_course()
            text_to_speech(response)

        elif "pesquisar" in command:
            if "google" in command:
                text_to_speech("O que você quer pesquisar no Google?")
                query = text_input_fallback()
                if query:
                    response = search_google(query)
                    text_to_speech(response)

            elif "wikipedia" in command:
                text_to_speech("O que você quer pesquisar na Wikipedia?")
                query = text_input_fallback()
                if query:
                    response = search_wikipedia(query)
                    text_to_speech(response)
            else:
                text_to_speech("Deseja pesquisar no Google ou na Wikipedia?")
                query = text_input_fallback()
                if query and ("google" in query):
                    text_to_speech("O que pesquisar no Google?")
                    search_query = text_input_fallback()
                    if search_query:
                        response = search_google(search_query)
                        text_to_speech(response)
                elif query and ("wikipedia" in query):
                    text_to_speech("O que pesquisar na Wikipedia?")
                    search_query = text_input_fallback()
                    if search_query:
                        response = search_wikipedia(search_query)
                        text_to_speech(response)
        else:
            text_to_speech("Comando não reconhecido. Tente novamente.")

# Tutorial rápido
print("🎤 COMO USAR:")
print("1. Grave seu comando de voz em qualquer app de gravação")
print("2. Salve como arquivo .wav ou .mp3")
print("3. Faça o upload quando solicitado")
print("4. Ou digite comandos manualmente")
print("\n🚀 Comandos disponíveis:")
print("• 'Abrir YouTube'")
print("• 'Onde tem Carrefour'")
print("• 'Curso de inteligência artificial'")
print("• 'Pesquisar no Google sobre [assunto]'")
print("• 'Pesquisar na Wikipedia sobre [assunto]'")
print("• 'Sair'")

# Executar a assistente virtual
virtual_assistant()


🎤 COMO USAR:
1. Grave seu comando de voz em qualquer app de gravação
2. Salve como arquivo .wav ou .mp3
3. Faça o upload quando solicitado
4. Ou digite comandos manualmente

🚀 Comandos disponíveis:
• 'Abrir YouTube'
• 'Onde tem Carrefour'
• 'Curso de inteligência artificial'
• 'Pesquisar no Google sobre [assunto]'
• 'Pesquisar na Wikipedia sobre [assunto]'
• 'Sair'



🎯 ESCOLHA UMA OPÇÃO:
1. 🎤 Upload de arquivo de áudio
2. ⌨️  Digitar comando manualmente
3. ❌ Sair
